In [ ]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup

In [ ]:
URL = "https://www.dogcatstar.com/blog/24hour-animal-hospital/?srsltid=AfmBOorT4sCf-7J95E1GuTg29XR3w-xYXl2qm-PZL1eOjPJ0k3ndzEem"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36 Edg/141.0.0.0"
}

In [ ]:
response = requests.get(URL, headers)
soup = BeautifulSoup(response.text, "html.parser")
# soup

In [ ]:
# 所有的資訊都在p tag裡面
p_tags = soup.select("p")
p_texts = [p.get_text(strip=True) for p in p_tags]

# 只擷取符合醫院的elements
p_texts = p_texts[3:9] + p_texts[13:51]

# 只留下名稱跟地址的資料
hospital_name_address = [item.split("電話：")[0] for item in p_texts]
hospital_name = [name.split("地址：")[0].strip() for name in hospital_name_address]
hospital_address = [address.split("地址：")[-1].strip() for address in hospital_name_address]

In [ ]:
# 將醫院名稱與地址放入DataFrame
df = pd.DataFrame(
    {"hospital_name": hospital_name, "hospital_address": hospital_address}
)

# 儲存row data
path = "../../data/raw/hospital_24_hr_data.csv"
os.makedirs(os.path.dirname(path), exist_ok=True)
df.to_csv(path, index=False)
print("原始CSV檔已儲存完畢")

In [ ]:
# 執行ETL
# 檢查是否有缺失值
df.isna().any()

# 對醫院名稱欄位的值做清理
df["hospital_name"] = df["hospital_name"].str.split("名稱：").str[-1].str.strip()

# 儲存ETL後的CSV檔
path = "../../data/processed/hospital_24_hr_data_ETL.csv"
os.makedirs(os.path.dirname(path), exist_ok=True)
df.to_csv(path, index=False)
print("ETL後的CSV檔已儲存完畢")